# Why does the output of dry for P100 plates differ from the output of the R code?

I've already shown that the output of dry for GCP plates is the same as the output of the R code (error on the order of 10^-16). But for P100 plates, the error seems to be on the order of 10^-2 sometimes. It can be bigger than that sometimes, though the average is about 10^-7. This is an attempt to dig into these discrepancies.

N.B. A particularly concerning thing is that the difference in values leads to a sample getting filtered out by dry once where the R code does not filter it out: P100, PRM, Plate 29, 6H.

In [112]:
# Location of dry results
gct_loc_dry = "/cmap/data/proteomics/dry/"

# Location of dry with no optimization results
gct_loc_dry_no_optim = "/cmap/data/proteomics/dry/dry_with_old_optim/"

# Location of R results
gct_loc_r_code = "/cmap/data/proteomics/produced_by_jjaffe_code/dry/wget_processed/"

# Location of unprocessed gcts
gct_loc_unprocessed = "/cmap/data/proteomics/harvest/wget_unprocessed/"

In [113]:
import numpy as np
import parse_gctoo as pg
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

# First, let's load in files and prove that the discrepancy exists; we'll use Plate 22 as an example
unpro_plate = pg.parse(gct_loc_unprocessed + "LINCS_P100_DIA_Plate22_annotated_minimized_2016-01-27_17-18-39.gct")
dry_plate = pg.parse(gct_loc_dry + "LINCS_P100_DIA_Plate22_annotated_minimized_2016-01-27_17-18-39.gct.dry.processed.gct")
r_plate = pg.parse(gct_loc_r_code + "LINCS_P100_DIA_Plate22_annotated_minimized_2016-01-27_17-18-39.processed.gct")

# dry
dry_plate.data_df.iloc[range(10), range(5)]

cid,PA5-11373-001A01,PA5-11373-002A02,PA5-11373-003A03,PA5-11373-004A04,PA5-11373-005A05
rid,,,,,
10011_DYRK_Y321_IYQY[+80]IQSR,-0.138658,0.124078,-0.191836,-0.160447,0.047043
1024_ISPK1_S369_TPKDS[+80]PGIPPSANAHQLFR,-0.213349,-0.545110,-0.511951,-0.197953,-0.210165
1078_ARM2_S87_RNS[+80]SEASSGDFLDLK,0.208837,1.080108,0.927835,0.065981,0.145001
1130_HSPC216_S321_LPLVPES[+80]PRR,0.566610,-0.300573,-0.228047,-0.249955,-0.117273
1142_CTG26_S956_ANAS[+80]PQKPLDLK,0.043985,0.317010,0.207650,0.030898,0.168164
1458_OCIA_S108_LENS[+80]PLGEALR,0.892273,-0.364768,0.463274,-0.159134,-0.927878
1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK,-0.255896,0.297277,0.091390,-0.084958,0.211983
151_ABI1_S184_TNPPTQKPPS[+80]PPMSGR,-0.385769,-0.200814,-0.196915,-0.318347,-0.111218
1797_WDR20_S465_SNS[+80]LPHSAVSNAGSK,-0.308875,0.703989,0.220569,-0.068506,0.280991


In [114]:
# R code
r_plate.data_df.iloc[range(10), range(5)]

cid,PA5-11373-001A01,PA5-11373-002A02,PA5-11373-003A03,PA5-11373-004A04,PA5-11373-005A05
rid,,,,,
10011_DYRK_Y321_IYQY[+80]IQSR,-0.138658,0.124078,-0.191836,-0.160447,0.047043
1024_ISPK1_S369_TPKDS[+80]PGIPPSANAHQLFR,-0.213349,-0.545110,-0.511951,-0.197953,-0.210165
1078_ARM2_S87_RNS[+80]SEASSGDFLDLK,0.208837,1.080108,0.927835,0.065981,0.145001
1130_HSPC216_S321_LPLVPES[+80]PRR,0.551021,-0.316162,-0.243636,-0.265544,-0.132862
1142_CTG26_S956_ANAS[+80]PQKPLDLK,0.030718,0.303743,0.194382,0.017631,0.154897
1458_OCIA_S108_LENS[+80]PLGEALR,0.892273,-0.364768,0.463274,-0.159134,-0.927878
1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK,-0.267365,0.285807,0.079921,-0.096427,0.200513
151_ABI1_S184_TNPPTQKPPS[+80]PPMSGR,-0.385769,-0.200814,-0.196915,-0.318347,-0.111218
1797_WDR20_S465_SNS[+80]LPHSAVSNAGSK,-0.308875,0.703989,0.220569,-0.068506,0.280991


In [115]:
dry_plate.data_df.iloc[range(10), range(5)] - r_plate.data_df.iloc[range(10), range(5)]

cid,PA5-11373-001A01,PA5-11373-002A02,PA5-11373-003A03,PA5-11373-004A04,PA5-11373-005A05
rid,,,,,
10011_DYRK_Y321_IYQY[+80]IQSR,-1.637579e-15,-3.469447e-16,-1.665335e-15,-2.303713e-15,-1.568190e-15
1024_ISPK1_S369_TPKDS[+80]PGIPPSANAHQLFR,3.275158e-15,1.776357e-15,4.440892e-16,5.662137e-15,9.436896e-16
1078_ARM2_S87_RNS[+80]SEASSGDFLDLK,-2.137179e-15,4.440892e-16,-1.998401e-15,-1.804112e-15,3.469447e-15
1130_HSPC216_S321_LPLVPES[+80]PRR,1.558898e-02,1.558898e-02,1.558898e-02,1.558898e-02,1.558898e-02
1142_CTG26_S956_ANAS[+80]PQKPLDLK,1.326758e-02,1.326758e-02,1.326758e-02,1.326758e-02,1.326758e-02
1458_OCIA_S108_LENS[+80]PLGEALR,-3.330669e-16,1.498801e-15,-4.718448e-15,-4.496403e-15,-2.109424e-15
1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK,1.146957e-02,1.146957e-02,1.146957e-02,1.146957e-02,1.146957e-02
151_ABI1_S184_TNPPTQKPPS[+80]PPMSGR,-1.665335e-16,-7.216450e-16,-1.165734e-15,-1.776357e-15,-1.554312e-15
1797_WDR20_S465_SNS[+80]LPHSAVSNAGSK,4.440892e-16,-1.110223e-15,6.661338e-16,-4.926615e-15,6.661338e-16


In [116]:
np.mean(dry_plate.data_df.iloc[range(10), range(5)] - r_plate.data_df.iloc[range(10), range(5)], axis=1)

rid
10011_DYRK_Y321_IYQY[+80]IQSR              -1.504352e-15
1024_ISPK1_S369_TPKDS[+80]PGIPPSANAHQLFR    2.420286e-15
1078_ARM2_S87_RNS[+80]SEASSGDFLDLK         -4.052314e-16
1130_HSPC216_S321_LPLVPES[+80]PRR           1.558898e-02
1142_CTG26_S956_ANAS[+80]PQKPLDLK           1.326758e-02
1458_OCIA_S108_LENS[+80]PLGEALR            -2.031708e-15
1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK     1.146957e-02
151_ABI1_S184_TNPPTQKPPS[+80]PPMSGR        -1.076916e-15
1797_WDR20_S465_SNS[+80]LPHSAVSNAGSK       -8.520962e-16
1811_MAP4_S2218_VGS[+80]LDNVGHLPAGGAVK     -1.387779e-16
dtype: float64

Across the whole probes, it looks like there are big differences for 3 out of the 10 probes: 1130_HSPC216_S321_LPLVPES[+80]PRR, 1142_CTG26_S956_ANAS[+80]PQKPLDLK, 1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK.

So, let's see how these probes differ from the other probes. Let's start with the unprocessed data.

In [117]:
# Define good and bad probes
bad_probes = ["1130_HSPC216_S321_LPLVPES[+80]PRR", "1142_CTG26_S956_ANAS[+80]PQKPLDLK", "1503_PDK1_S241_ANS[+80]FVGTAQYVSPELLTEK"]
good_probes = ['10011_DYRK_Y321_IYQY[+80]IQSR', '1024_ISPK1_S369_TPKDS[+80]PGIPPSANAHQLFR', '1078_ARM2_S87_RNS[+80]SEASSGDFLDLK', '12_KIAA0701_S935_SMS[+80]VDLSHIPLKDPLLFK', '1458_OCIA_S108_LENS[+80]PLGEALR', '151_ABI1_S184_TNPPTQKPPS[+80]PPMSGR', '1797_WDR20_S465_SNS[+80]LPHSAVSNAGSK']

In [118]:
# Unprocessed
traces = []
for probe in bad_probes:
    this_data = unpro_plate.data_df.ix[probe, :].values.flatten()
    trace = go.Scatter(
        x = unpro_plate.data_df.columns.values,
        y = this_data,
        mode = 'lines',
        name = probe,
        line = dict(
            color = ('rgb(205, 12, 24)')
        )
    )
    traces.append(trace)
    
for probe in good_probes:
    this_data = unpro_plate.data_df.ix[probe, :].values.flatten()
    trace = go.Scatter(
        x = unpro_plate.data_df.columns.values,
        y = this_data,
        mode = 'lines',
        name = probe,
        line = dict(
            color = ('rgb(22, 96, 167)')
        )
    )
    traces.append(trace)

# Plot
py.iplot(traces, filename='first_10_probes_unpro')

The H07, H08, H09 samples look pretty weird. What are those samples?

In [119]:
weird_samples = [89, 90, 91]

unpro_plate.col_metadata_df.ix[weird_samples, "pert_iname"]

cid
PA5-11373-091H07    Okadaic Acid
PA5-11373-092H08    Okadaic Acid
PA5-11373-093H09    Okadaic Acid
Name: pert_iname, dtype: object

Turns out it's just okadaic acid. Maybe it's killing the cells. Let's not worry about it.

Now, let's look at how the 2 scripts (dry v. R) try to correct this.

In [120]:
# Dry
bad_dry_traces = []
good_dry_traces = []

for probe in bad_probes:
    try:
        this_data = dry_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = dry_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = probe,
            line = dict(
                color = ('rgb(205, 12, 24)')
            )
        )
        bad_dry_traces.append(trace)
    except:
        print("{} not in dry data.".format(probe))

for probe in good_probes:
    try:
        this_data = dry_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = dry_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = probe,
            line = dict(
                color = ('rgb(22, 96, 167)')
            )
        )
        good_dry_traces.append(trace)
    except:
        print("{} not in dry data.".format(probe))
        
dry_traces = bad_dry_traces + good_dry_traces

12_KIAA0701_S935_SMS[+80]VDLSHIPLKDPLLFK not in dry data.


In [121]:
# R
bad_r_traces = []
good_r_traces = []

for probe in bad_probes:
    try:
        this_data = r_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = r_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = probe,
            line = dict(
                color = ('rgb(205, 12, 24)')
            )
        )
        bad_r_traces.append(trace)
    except:
        print("{} not in r data.".format(probe))

for probe in good_probes:
    try:
        this_data = r_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = r_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = probe,
            line = dict(
                color = ('rgb(22, 96, 167)')
            )
        )
        good_r_traces.append(trace)
    except:
        print("{} not in r data.".format(probe))
        
r_traces = bad_r_traces + good_r_traces

12_KIAA0701_S935_SMS[+80]VDLSHIPLKDPLLFK not in r data.


In [122]:
# Dry
dry_layout = go.Layout(
    title='dry')
dry_fig = go.Figure(data=dry_traces, layout=dry_layout)
py.iplot(dry_fig, filename='first_10_probes_dry')

In [123]:
# R
r_layout = go.Layout(
    title='R')
r_fig = go.Figure(data=r_traces, layout=r_layout)
py.iplot(r_fig, filename='first_10_probes_r')

Not super helpful. Let's compare the data from dry and R for the bad probes directly.

In [124]:
# Bad probes only
bad_probe_traces = []
for probe in bad_probes:
    this_data = unpro_plate.data_df.ix[probe, :].values.flatten()
    trace = go.Scatter(
        x = unpro_plate.data_df.columns.values,
        y = this_data,
        mode = 'lines',
        name = probe,
        line = dict(
            color = ('rgb(205, 12, 24)')
        )
    )
    bad_probe_traces.append(trace)
    
    try:
        this_data = dry_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = dry_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = "DRY_" + probe,
            line = dict(
                color = ('rgb(255, 165, 0)')
            )
        )
        bad_probe_traces.append(trace)
    except:
        print("{} not in dry data.".format(probe))
        
    try:
        this_data = r_plate.data_df.ix[probe, :].values.flatten()
        trace = go.Scatter(
            x = r_plate.data_df.columns.values,
            y = this_data,
            mode = 'lines',
            name = "R_" + probe,
            line = dict(
                color = ('rgb(0, 100, 0)')
            )
        )
        bad_probe_traces.append(trace)
    except:
        print("{} not in r data.".format(probe))
        
bad_only_layout = go.Layout(
    title='Bad probes only')
bad_probe_fig = go.Figure(data=bad_probe_traces, layout=bad_only_layout)
py.iplot(bad_probe_fig, filename='bad_probes_only_probes')


Okay, so we can see the error for the probes is not caused by MOST of the values being off, but just for a subset. The next step might be to look into those samples for which the difference is big, cause it's really just those that are the problem.

In [125]:
bad_samples = ["PA5-11373-091H07", "PA5-11373-092H08", "PA5-11373-093H09"]

unpro_trace = go.Scatter(
        x = unpro_plate.data_df.index.values,
        y = unpro_plate.data_df.loc[:, bad_samples[0]],
        mode = 'lines',
        name = "unpro",
)
dry_trace = go.Scatter(
        x = dry_plate.data_df.index.values,
        y = dry_plate.data_df.loc[:, bad_samples[0]],
        mode = 'lines',
        name = "dry",
)
r_trace = go.Scatter(
        x = r_plate.data_df.index.values,
        y = r_plate.data_df.loc[:, bad_samples[0]],
        mode = 'lines',
        name = "R",
)

bad_sample_layout = go.Layout(
    title='Sample {}'.format(bad_samples[0]))
bad_sample_fig = go.Figure(data=[unpro_trace, dry_trace, r_trace], layout=bad_sample_layout)
py.iplot(bad_sample_fig, filename='bad_sample')

Green (R) is almost always higher. Well, is it "correct"? We can evaluate this by looking at the residuals, or the distance from each value to its probe median.

In [126]:
dry_probe_medians = np.nanmedian(dry_plate.data_df, axis=1)
r_probe_medians = np.nanmedian(r_plate.data_df, axis=1)

bad_sample1_dry_resids = dry_plate.data_df.loc[:, bad_samples[0]] - dry_probe_medians
bad_sample1_r_resids = r_plate.data_df.loc[:, bad_samples[0]] - r_probe_medians

print("bad_sample1_dry_resids: {}".format(np.mean(bad_sample1_dry_resids)))
print("bad_sample1_r_resids: {}".format(np.mean(bad_sample1_r_resids)))


bad_sample1_dry_resids: -1.04800541251
bad_sample1_r_resids: -0.0209733431959


** Need to look at the medians BEFORE row median normalization. Looking at it now isn't too informative (all of them are right around 0, for both dry and R).

The goal of applying offsets is to get each value as close as possible to the probe median. Let's see which method does this better!